<a href="https://colab.research.google.com/github/SaharshKhicha18/Solubility-Prediction-python/blob/main/CompDrugDiscovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Conda **

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

**Importing rdkit library**

In [ ]:
! conda install -c rdkit rdkit -y

In [ ]:
import rdkit 

**Dataset**

In [ ]:
! wget https://pubs.acs.org/doi/suppl/10.1021/ci034243x/suppl_file/ci034243xsi20040112_053635.txt

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('ci034243xsi20040112_053635.txt')
data

**Simplified Molecular Input-Line Entry System data **

In [ ]:
data.SMILES

**Convert SMILES to rdkit object**

In [ ]:
from rdkit import Chem

In [ ]:
rdlist = []
for i in data.SMILES:
  x= Chem.MolFromSmiles(i)
  rdlist.append(x)


**Calculation of the Molecular Descriptors used in the study**

To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:

cLogP (Octanol-water partition coefficient)

MW (Molecular weight)

RB (Number of rotatable bonds)

AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)

Unfortunately, rdkit readily computes the first 3. AP descriptor will be computed manually by the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute.

In [ ]:
import numpy as np
from rdkit.Chem import Descriptors

In [ ]:
def generate(smiles, verbose=False):

    moldescdata= [] #convert the smiles into rdkit object
    for i in smiles:
        x=Chem.MolFromSmiles(i) 
        moldescdata.append(x)
       
    baseData= np.arange(1,1)
    j=0  
    for x in moldescdata:        
       
        d_MolLogP = Descriptors.MolLogP(x)
        d_MolWt = Descriptors.MolWt(x)
        d_NumRotatableBonds = Descriptors.NumRotatableBonds(x)
           
        row = np.array([d_MolLogP, d_MolWt, d_NumRotatableBonds])   #numpy array for the arrangement of 3 MD
    
        if(j==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        j = j+1     
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames) #pandas data frame 
    
    return descriptors

In [ ]:
desc3 = generate(data.SMILES)
desc3

**Molecular Descriptor Aromatic Proportion (AP)**

**  (a) Number of Aromatic Atoms **

In [ ]:
def AromaticAtoms(x):
  is_aromatic = [x.GetAtomWithIdx(i).GetIsAromatic() for i in range(x.GetNumAtoms())]
  count = 0
  for i in is_aromatic:
    if i == True:
      count =count +1
  return count


In [ ]:
d_aromaticatoms = []
for i in rdlist:
  d_aromaticatoms.append(AromaticAtoms(i))
d_aromaticatoms

**  (b) Number of heavy atoms**

In [ ]:
d_heavyatoms = []
for i in rdlist:
  d_heavyatoms.append(Descriptors.HeavyAtomCount(i))
d_heavyatoms

**  AP = AromaticAtoms/HeavyAtoms**

In [ ]:
d_AP = []
for i in rdlist:
  d_AP.append(AromaticAtoms(i)/Descriptors.HeavyAtomCount(i))
d_AP

**Now that we have calculated all the 4 descriptors in order to predict the solubility, we will concatenate all the descriptors data in one table**

In [ ]:
df_d_AP = pd.DataFrame(d_AP,columns = ['AromaticProportion'])
df_d_AP

In [ ]:
X_desc4 = pd.concat([desc3,df_d_AP], axis = 1)
X_desc4

**Y Matrix (that has to be predicted by the X matrix values)**

In [ ]:
Y = data.iloc[:,1]
Y

**Data Spliting and generating a linear regression model**

   (A) First Train and Test split and linear regression 
   
   (B) Second Full dataset linear regression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_desc4, Y, test_size=0.3) #30% test size

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
regressmodel = linear_model.LinearRegression()
regressmodel.fit(Xtrain, Ytrain)

Prediction of the Xtrain - For comparitive purposes

In [ ]:
YPredicttrain = regressmodel.predict(Xtrain)
print("Coefficients = ", regressmodel.coef_)
print("Intercept = ", regressmodel.intercept_)
print("Mean squared error = %.3f" % mean_squared_error(Ytrain, YPredicttrain))
print("Pearsons Correlation coeffcient squared (R^2) = %.3f" % r2_score(Ytrain, YPredicttrain)) #Coeffcient of determination

Prediction of Xtest - for comparitive purposes

In [ ]:
YPredicttest = regressmodel.predict(Xtest)
print("Coefficients = " + regressmodel.coef_)
print("Intercept = " + regressmodel.intercept_)
print("Mean squared error = %.3f" % mean_squared_error(Ytest, YPredicttest))
print("Pearsons Correlation coeffcient squared (R^2) = %.3f" % r2_score(Ytest, YPredicttest)) #Coeffcient of determination

# Linear regression Equation

Full Dataset model

LogS = 0.22 (Y-intercept) - 0.72LogP - 0.0069MW + 0.0158RB - 0.428AP

4 Descriptors and the numbers are the regression coeffcients

In [ ]:
print("LogS = %.3f %.3f LogP %.3f MW %.3f RB %.3f AP" %(regressmodel.intercept_, regressmodel.coef_[0], regressmodel.coef_[1], regressmodel.coef_[2], regressmodel.coef_[3]))

Scatter plot of the predicted value vs the experimental value

In [ ]:
import matplotlib.pyplot as plot

In [ ]:
Ytrain.shape, YPredicttrain.shape, Ytest.shape, YPredicttest.shape

# Horizontal Plot

In [ ]:
plot.figure(figsize = (12, 6))

#Plot 1
plot.subplot(1, 2, 1) #2 rows for 2 graphs, 1 column and first graph
plot.scatter(x = Ytrain, y = YPredicttrain, c = "#00FF13", alpha = 0.2)

#Add trendline
fit = np.polyfit(Ytrain, YPredicttrain, 1)
trend = np.poly1d(fit)
plot.plot(Ytest, trend(Ytest), "#FF2D00")

#Add axis labels
plot.xlabel("Experimental LogS")
plot.ylabel("Predicted LogS")

#Plot 2
plot.subplot(1, 2, 2)
plot.scatter(x = Ytest, y = YPredicttest, c = "#F000FF", alpha = 0.2)

fit = np.polyfit(Ytest, YPredicttest, 1)
trend = np.poly1d(fit)
plot.plot(Ytest, trend(Ytest), "#FF2D00")

plot.xlabel("Experimental LogS")
plot.ylabel("Predicted LogS")

plot.savefig("Verticalplot.pdf")
plot.show()